<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №2</center>

<center>PySpark»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>

<br>
<p style="text-align:right;">Обучающийся: Скляров Д.В.
<br>6131-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2024</center>


In [ ]:
import warnings
from datetime import datetime, date
import pyspark.pandas as ps
from pyspark.sql import (
    SparkSession,
    Row
)
from pyspark.sql import Column
from pyspark.sql.functions import (
    upper,
    expr
)
import numpy as np
import pandas as pd
from pyspark.sql.functions import pandas_udf
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
# Ignore warnings coming from Arrow optimizations.
warnings.filterwarnings("ignore")

#### DataFrame


In [4]:
spark = SparkSession.builder.getOrCreate()
df = spark.createDataFrame([
    Row(a=1, b=2, c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3, c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=3, b=5, c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
])
df

DataFrame[a: bigint, b: bigint, c: string, d: date, e: timestamp]

In [7]:
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|  2|string1|2000-01-01|2000-01-01 12:00:00|
|  2|  3|string2|2000-02-01|2000-01-02 12:00:00|
|  3|  5|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [8]:
df.show(1)

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|  2|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+
only showing top 1 row



In [9]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

a,b,c,d,e
1,2,string1,2000-01-01,2000-01-01 12:00:00
2,3,string2,2000-02-01,2000-01-02 12:00:00
3,5,string3,2000-03-01,2000-01-03 12:00:00


In [10]:
df.show(1, vertical=True)

-RECORD 0------------------
 a   | 1                   
 b   | 2                   
 c   | string1             
 d   | 2000-01-01          
 e   | 2000-01-01 12:00:00 
only showing top 1 row



In [11]:
df.columns

['a', 'b', 'c', 'd', 'e']

In [12]:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: long (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [13]:
df.select("a", "b", "c").describe().show()

+-------+---+------------------+-------+
|summary|  a|                 b|      c|
+-------+---+------------------+-------+
|  count|  3|                 3|      3|
|   mean|2.0|3.3333333333333335|   NULL|
| stddev|1.0|1.5275252316519468|   NULL|
|    min|  1|                 2|string1|
|    max|  3|                 5|string3|
+-------+---+------------------+-------+



In [14]:
df.collect()

[Row(a=1, b=2, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0)),
 Row(a=2, b=3, c='string2', d=datetime.date(2000, 2, 1), e=datetime.datetime(2000, 1, 2, 12, 0)),
 Row(a=3, b=5, c='string3', d=datetime.date(2000, 3, 1), e=datetime.datetime(2000, 1, 3, 12, 0))]

In [15]:
df.take(1)

[Row(a=1, b=2, c='string1', d=datetime.date(2000, 1, 1), e=datetime.datetime(2000, 1, 1, 12, 0))]

In [16]:
df.toPandas()

,a,b,c,d,e
0,1,2,string1,2000-01-01,2000-01-01 12:00:00
1,2,3,string2,2000-02-01,2000-01-02 12:00:00
2,3,5,string3,2000-03-01,2000-01-03 12:00:00


In [17]:
df.a

Column<'a'>

In [19]:
type(df.c) == type(upper(df.c)) == type(df.c.isNull())

True

In [20]:
df.select(df.c).show()

+-------+
|      c|
+-------+
|string1|
|string2|
|string3|
+-------+



In [21]:
df.withColumn('upper_c', upper(df.c)).show()

+---+---+-------+----------+-------------------+-------+
|  a|  b|      c|         d|                  e|upper_c|
+---+---+-------+----------+-------------------+-------+
|  1|  2|string1|2000-01-01|2000-01-01 12:00:00|STRING1|
|  2|  3|string2|2000-02-01|2000-01-02 12:00:00|STRING2|
|  3|  5|string3|2000-03-01|2000-01-03 12:00:00|STRING3|
+---+---+-------+----------+-------------------+-------+



In [22]:
df.filter(df.a == 1).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|  2|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [ ]:
@pandas_udf('long')
def pandas_plus_one(series: pd.Series) -> pd.Series:
    # Simply plus one by using pandas Series.
    return series + 1


df.select(pandas_plus_one(df.a)).show()

+------------------+
|pandas_plus_one(a)|
+------------------+
|                 2|
|                 3|
|                 4|
+------------------+



In [ ]:
def pandas_filter_func(iterator):
    for pandas_df in iterator:
        yield pandas_df[pandas_df.a == 1]


df.mapInPandas(pandas_filter_func, schema=df.schema).show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|  2|string1|2000-01-01|2000-01-01 12:00:00|
+---+---+-------+----------+-------------------+



In [27]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
| blue|banana|  2| 20|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
|  red|carrot|  5| 50|
|black|carrot|  6| 60|
|  red|banana|  7| 70|
|  red| grape|  8| 80|
+-----+------+---+---+



In [28]:
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [ ]:
def plus_mean(pandas_df):
    return pandas_df.assign(v1=pandas_df.v1 - pandas_df.v1.mean())


df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  0| 60|
| blue|banana| -1| 20|
| blue| grape|  1| 40|
|  red|banana| -3| 10|
|  red|carrot| -1| 30|
|  red|carrot|  0| 50|
|  red|banana|  2| 70|
|  red| grape|  3| 80|
+-----+------+---+---+



In [ ]:
df1 = spark.createDataFrame(
    [(20000101, 1, 1.0), (20000101, 2, 2.0),
     (20000102, 1, 3.0), (20000102, 2, 4.0)],
    ('time', 'id', 'v1'))

df2 = spark.createDataFrame(
    [(20000101, 1, 'x'), (20000101, 2, 'y')],
    ('time', 'id', 'v2'))


def merge_ordered(l, r):
    return pd.merge_ordered(l, r)


df1.groupby('id').cogroup(df2.groupby('id')).applyInPandas(
    merge_ordered, schema='time int, id int, v1 double, v2 string').show()

+--------+---+---+----+
|    time| id| v1|  v2|
+--------+---+---+----+
|20000101|  1|1.0|   x|
|20000102|  1|3.0|NULL|
|20000101|  2|2.0|   y|
|20000102|  2|4.0|NULL|
+--------+---+---+----+



In [31]:
df.write.csv('foo.csv', header=True)
spark.read.csv('foo.csv', header=True).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  6| 60|
| blue|banana|  2| 20|
| blue| grape|  4| 40|
|  red|carrot|  3| 30|
|  red|banana|  7| 70|
|  red|banana|  1| 10|
|  red|carrot|  5| 50|
|  red| grape|  8| 80|
+-----+------+---+---+



In [32]:
df.write.parquet('bar.parquet')
spark.read.parquet('bar.parquet').show()

24/11/26 02:40:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/11/26 02:40:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/11/26 02:40:00 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  6| 60|
| blue|banana|  2| 20|
|  red|carrot|  5| 50|
| blue| grape|  4| 40|
|  red|banana|  7| 70|
|  red|banana|  1| 10|
|  red|carrot|  3| 30|
|  red| grape|  8| 80|
+-----+------+---+---+



In [33]:
df.write.orc('zoo.orc')
spark.read.orc('zoo.orc').show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  7| 70|
|  red| grape|  8| 80|
|black|carrot|  6| 60|
| blue|banana|  2| 20|
|  red|banana|  1| 10|
|  red|carrot|  5| 50|
|  red|carrot|  3| 30|
| blue| grape|  4| 40|
+-----+------+---+---+



In [34]:
df.createOrReplaceTempView("tableA")
spark.sql("SELECT count(*) from tableA").show()

+--------+
|count(1)|
+--------+
|       8|
+--------+



In [ ]:
@pandas_udf("integer")
def add_one(s: pd.Series) -> pd.Series:
    return s + 1


spark.udf.register("add_one", add_one)
spark.sql("SELECT add_one(v1) FROM tableA").show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+



In [37]:
df.selectExpr('add_one(v1)').show()
df.select(expr('count(*)') > 0).show()

+-----------+
|add_one(v1)|
+-----------+
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
+-----------+

+--------------+
|(count(1) > 0)|
+--------------+
|          true|
+--------------+



#### Pandas API on Spark


In [38]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])
psdf = ps.DataFrame(
    {
        'a': [1, 2, 3, 4, 5, 6],
        'b': [100, 200, 300, 400, 500, 600],
        'c': [
            "one",
            'two',
            'three',
            'four',
            'five',
            'six'
        ]
    },
    index=[10, 20, 30, 40, 50, 60]
)
psdf

,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


In [40]:
dates = pd.date_range('20130101', periods=6)
dates

DatetimeIndex(['2013-01-01', '2013-01-02', '2013-01-03', '2013-01-04',
               '2013-01-05', '2013-01-06'],
              dtype='datetime64[ns]', freq='D')

In [41]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
pdf

,A,B,C,D
2013-01-01,-1.910374,0.835073,-1.115793,1.780395
2013-01-02,-0.853076,-0.840271,0.438799,1.192484
2013-01-03,-2.286823,0.566741,3.047740,0.246229
2013-01-04,0.316905,-0.060914,-0.230360,0.534698
2013-01-05,-0.937877,-0.890546,-0.411935,-0.260814
2013-01-06,-0.507251,-0.004596,0.017531,-0.320394


In [42]:
psdf = ps.from_pandas(pdf)
type(psdf)

pyspark.pandas.frame.DataFrame

In [43]:
sdf = spark.createDataFrame(pdf)
sdf.show()

+-------------------+--------------------+--------------------+-------------------+
|                  A|                   B|                   C|                  D|
+-------------------+--------------------+--------------------+-------------------+
|-1.9103744635881437|  0.8350725664394424| -1.1157933554035884| 1.7803947396201527|
|-0.8530762409271604| -0.8402706774599453| 0.43879851032281086| 1.1924842874409418|
| -2.286822873697785|  0.5667414356739335|   3.047740157070994| 0.2462291591928036|
| 0.3169049118748236|-0.06091395118068523|-0.23036042311685842| 0.5346980866950564|
|-0.9378772304594643| -0.8905463294904983| -0.4119345598874601| -0.260813654382047|
| -0.507251344189168|-0.00459556164906...|0.017531420154260256|-0.3203938279997675|
+-------------------+--------------------+--------------------+-------------------+



In [44]:
psdf = sdf.pandas_api()
psdf

,A,B,C,D
0,-1.910374,0.835073,-1.115793,1.780395
1,-0.853076,-0.840271,0.438799,1.192484
2,-2.286823,0.566741,3.047740,0.246229
3,0.316905,-0.060914,-0.230360,0.534698
4,-0.937877,-0.890546,-0.411935,-0.260814
5,-0.507251,-0.004596,0.017531,-0.320394


In [45]:
psdf.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

In [46]:
psdf.head()

,A,B,C,D
0,-1.910374,0.835073,-1.115793,1.780395
1,-0.853076,-0.840271,0.438799,1.192484
2,-2.286823,0.566741,3.047740,0.246229
3,0.316905,-0.060914,-0.230360,0.534698
4,-0.937877,-0.890546,-0.411935,-0.260814


In [47]:
psdf.index

Index([0, 1, 2, 3, 4, 5], dtype='int64')

In [48]:
psdf.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [49]:
psdf.to_numpy()

/home/nia/.pyenv/versions/3.10.14/envs/ssau3.10/lib/python3.10/site-packages/pyspark/pandas/utils.py:1016: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


array([[-1.91037446,  0.83507257, -1.11579336,  1.78039474],
       [-0.85307624, -0.84027068,  0.43879851,  1.19248429],
       [-2.28682287,  0.56674144,  3.04774016,  0.24622916],
       [ 0.31690491, -0.06091395, -0.23036042,  0.53469809],
       [-0.93787723, -0.89054633, -0.41193456, -0.26081365],
       [-0.50725134, -0.00459556,  0.01753142, -0.32039383]])

In [50]:
psdf.describe()

24/11/26 02:43:03 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,A,B,C,D
count,6.000000,6.000000,6.000000,6.000000
mean,-1.029750,-0.065752,0.290997,0.528766
std,0.946770,0.706465,1.445078,0.828428
min,-2.286823,-0.890546,-1.115793,-0.320394
25%,-1.910374,-0.840271,-0.411935,-0.260814
50%,-0.937877,-0.060914,-0.230360,0.246229
75%,-0.507251,0.566741,0.438799,1.192484
max,0.316905,0.835073,3.047740,1.780395


In [51]:
psdf.T

,0,1,2,3,4,5
A,-1.910374,-0.853076,-2.286823,0.316905,-0.937877,-0.507251
B,0.835073,-0.840271,0.566741,-0.060914,-0.890546,-0.004596
C,-1.115793,0.438799,3.047740,-0.230360,-0.411935,0.017531
D,1.780395,1.192484,0.246229,0.534698,-0.260814,-0.320394


In [52]:
psdf.sort_index(ascending=False)

,A,B,C,D
5,-0.507251,-0.004596,0.017531,-0.320394
4,-0.937877,-0.890546,-0.411935,-0.260814
3,0.316905,-0.060914,-0.230360,0.534698
2,-2.286823,0.566741,3.047740,0.246229
1,-0.853076,-0.840271,0.438799,1.192484
0,-1.910374,0.835073,-1.115793,1.780395


In [53]:
psdf.sort_values(by='B')

,A,B,C,D
4,-0.937877,-0.890546,-0.411935,-0.260814
1,-0.853076,-0.840271,0.438799,1.192484
3,0.316905,-0.060914,-0.230360,0.534698
5,-0.507251,-0.004596,0.017531,-0.320394
2,-2.286823,0.566741,3.047740,0.246229
0,-1.910374,0.835073,-1.115793,1.780395


In [54]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])
pdf1.loc[dates[0]:dates[1], 'E'] = 1
psdf1 = ps.from_pandas(pdf1)
psdf1

,A,B,C,D,E
2013-01-01,-1.910374,0.835073,-1.115793,1.780395,1.0
2013-01-02,-0.853076,-0.840271,0.438799,1.192484,1.0
2013-01-03,-2.286823,0.566741,3.047740,0.246229,NaN
2013-01-04,0.316905,-0.060914,-0.230360,0.534698,NaN


In [55]:
psdf1.dropna(how='any')

,A,B,C,D,E
2013-01-01,-1.910374,0.835073,-1.115793,1.780395,1.0
2013-01-02,-0.853076,-0.840271,0.438799,1.192484,1.0


In [56]:
psdf1.fillna(value=5)

,A,B,C,D,E
2013-01-01,-1.910374,0.835073,-1.115793,1.780395,1.0
2013-01-02,-0.853076,-0.840271,0.438799,1.192484,1.0
2013-01-03,-2.286823,0.566741,3.047740,0.246229,5.0
2013-01-04,0.316905,-0.060914,-0.230360,0.534698,5.0


In [57]:
psdf.mean()

A   -1.029750
B   -0.065752
C    0.290997
D    0.528766
dtype: float64

In [ ]:
# Keep its default value.
prev = spark.conf.get("spark.sql.execution.arrow.pyspark.enabled")
# Use default index prevent overhead.
ps.set_option("compute.default_index_type", "distributed")

In [60]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", True)
%timeit ps.range(300000).to_pandas()

69.2 ms ± 7.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", False)
%timeit ps.range(300000).to_pandas()

702 ms ± 23.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
ps.reset_option("compute.default_index_type")
# Set its default value back.
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", prev)

In [ ]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                     'C': np.random.randn(8),
                     'D': np.random.randn(8)})
psdf

,A,B,C,D
0,foo,one,0.481138,0.035343
1,bar,one,-0.301362,1.030881
2,foo,two,0.630108,0.202203
3,bar,three,-0.291246,-0.011235
4,foo,two,0.563144,-0.313861
5,bar,two,0.718481,0.340947
6,foo,one,-0.560755,-1.689515
7,foo,three,0.004668,-0.305274


In [64]:
psdf.groupby('A').sum()

,C,D
A,,
foo,1.118304,-2.071105
bar,0.125872,1.360593


In [65]:
psdf.groupby(['A', 'B']).sum()

C         D
A   B                        
foo one   -0.079616 -1.654173
bar one   -0.301362  1.030881
foo two    1.193252 -0.111658
bar three -0.291246 -0.011235
    two    0.718481  0.340947
foo three  0.004668 -0.305274

In [66]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [69]:
psser = ps.Series(pser)
psser

2000-01-01   -0.263948
2000-01-02    1.895962
2000-01-03   -0.457679
2000-01-04   -0.407413
2000-01-05   -0.974374
2000-01-06   -0.818948
2000-01-07   -0.460600
2000-01-08   -0.461209
2000-01-09    1.041917
2000-01-10    2.076980
2000-01-11    1.043500
2000-01-12    1.634154
2000-01-13    0.871683
2000-01-14   -2.343384
2000-01-15    0.736418
2000-01-16    0.448460
2000-01-17   -1.509008
2000-01-18   -0.057329
2000-01-19   -2.017794
2000-01-20    0.332325
2000-01-21   -0.630622
2000-01-22    1.253462
2000-01-23    1.069643
2000-01-24   -0.281925
2000-01-25    0.343444
2000-01-26    0.939437
2000-01-27    0.597569
2000-01-28   -0.161748
2000-01-29    0.527559
2000-01-30    1.106155
2000-01-31   -1.677163
2000-02-01    1.599087
2000-02-02    0.178165
2000-02-03   -0.231503
2000-02-04    0.882467
2000-02-05   -1.027145
2000-02-06    0.035458
2000-02-07    0.549862
2000-02-08    0.886972
2000-02-09   -0.360444
2000-02-10   -0.110285
2000-02-11   -0.518263
2000-02-12    0.385645
2000-02-13 

In [70]:
psser = psser.cummax()
psser

24/11/26 02:46:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


2000-01-01   -0.263948
2000-01-02    1.895962
2000-01-03    1.895962
2000-01-04    1.895962
2000-01-05    1.895962
2000-01-06    1.895962
2000-01-07    1.895962
2000-01-08    1.895962
2000-01-09    1.895962
2000-01-10    2.076980
2000-01-11    2.076980
2000-01-12    2.076980
2000-01-13    2.076980
2000-01-14    2.076980
2000-01-15    2.076980
2000-01-16    2.076980
2000-01-17    2.076980
2000-01-18    2.076980
2000-01-19    2.076980
2000-01-20    2.076980
2000-01-21    2.076980
2000-01-22    2.076980
2000-01-23    2.076980
2000-01-24    2.076980
2000-01-25    2.076980
2000-01-26    2.076980
2000-01-27    2.076980
2000-01-28    2.076980
2000-01-29    2.076980
2000-01-30    2.076980
2000-01-31    2.076980
2000-02-01    2.076980
2000-02-02    2.076980
2000-02-03    2.076980
2000-02-04    2.076980
2000-02-05    2.076980
2000-02-06    2.076980
2000-02-07    2.076980
2000-02-08    2.076980
2000-02-09    2.076980
2000-02-10    2.076980
2000-02-11    2.076980
2000-02-12    2.076980
2000-02-13 

In [71]:
psser.plot()

24/11/26 02:46:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:12 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [72]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])
psdf = ps.from_pandas(pdf)
psdf = psdf.cummax()
psdf.plot()

24/11/26 02:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:35 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [73]:
psdf.to_csv('foo.csv')
ps.read_csv('foo.csv').head(10)

24/11/26 02:46:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,0.181035,1.525859,1.826293,-0.911967
1,0.221405,1.525859,1.826293,-0.165818
2,0.886915,1.525859,1.826293,-0.165818
3,2.798992,1.525859,1.826293,0.903961
4,2.798992,1.525859,1.826293,0.903961
5,2.798992,1.525859,1.826293,0.903961
6,2.798992,1.525859,1.826293,1.193734
7,2.798992,1.525859,1.826293,2.562865
8,2.798992,1.525859,1.826293,2.562865
9,2.798992,1.525859,1.826293,2.562865


In [74]:
psdf.to_parquet('bar.parquet')
ps.read_parquet('bar.parquet').head(10)

24/11/26 02:46:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,0.181035,1.525859,1.826293,-0.911967
1,0.221405,1.525859,1.826293,-0.165818
2,0.886915,1.525859,1.826293,-0.165818
3,2.798992,1.525859,1.826293,0.903961
4,2.798992,1.525859,1.826293,0.903961
5,2.798992,1.525859,1.826293,0.903961
6,2.798992,1.525859,1.826293,1.193734
7,2.798992,1.525859,1.826293,2.562865
8,2.798992,1.525859,1.826293,2.562865
9,2.798992,1.525859,1.826293,2.562865


In [75]:
psdf.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)

24/11/26 02:46:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/26 02:46:57 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,A,B,C,D
0,0.181035,1.525859,1.826293,-0.911967
1,0.221405,1.525859,1.826293,-0.165818
2,0.886915,1.525859,1.826293,-0.165818
3,2.798992,1.525859,1.826293,0.903961
4,2.798992,1.525859,1.826293,0.903961
5,2.798992,1.525859,1.826293,0.903961
6,2.798992,1.525859,1.826293,1.193734
7,2.798992,1.525859,1.826293,2.562865
8,2.798992,1.525859,1.826293,2.562865
9,2.798992,1.525859,1.826293,2.562865
